### 라이브러리 불러오기

In [1]:
!pip install playsound

In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from playsound import playsound
import numpy as np
import cv2

### 모델 정의

In [3]:
# 얼굴 찾는 모델
facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
# 마스크 검출 모델
model = load_model('models/mask_detector.model')

error: OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1121: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "models/deploy.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'


### 실시간 웹캠 출력

In [20]:
# 실시간 웹캠 읽기
cap = cv2.VidioCapture(0)

# 캡쳐 프레임 사이즈 조절
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
i = 0

AttributeError: module 'cv2.cv2' has no attribute 'VidioCapture'

### 소리 확인

In [ ]:
playsound('wear_mask.mp3')

In [ ]:
playsound('bbi.mp3')

### 전처리 및 웹캠 이용한 마스크 착용여부 출력

In [11]:
while cap.isOpened():
    
    time_tick = 0
    
    ret, img = cap.read()
    if not ret:
        break
    
    # 이미지의 높이와 너비 추출
    h, w = img.shape[:2]

    # 이미지 전처리
    # ref. https://www.pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/
    blob = cv2.dnn.blobFromImage(img, scalefactor=1., size=(300, 300), mean=(104., 177., 123.))

    # facenet의 input으로 blob을 설정
    facenet.setInput(blob)
    # facenet 결과 추론, 얼굴 추출 결과가 dets의 저장
    dets = facenet.forward()

    # 한 프레임 내의 여러 얼굴들을 받음
    result_img = img.copy()

    # 마스크를 찾용했는지 확인
    for i in range(dets.shape[2]):

        # 검출한 결과가 신뢰도
        confidence = dets[0, 0, i, 2]
        # 신뢰도를 0.5로 임계치 지정
        if confidence < 0.5:
            continue

        # 바운딩 박스를 구함
        x1 = int(dets[0, 0, i, 3] * w)
        y1 = int(dets[0, 0, i, 4] * h)
        x2 = int(dets[0, 0, i, 5] * w)
        y2 = int(dets[0, 0, i, 6] * h)

        # 원본 이미지에서 얼굴영역 추출
        face = img[y1:y2, x1:x2]

        # 추출한 얼굴영역을 전처리
        face_input = cv2.resize(face, dsize=(224, 224))
        face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
        face_input = preprocess_input(face_input)
        face_input = np.expand_dims(face_input, axis=0)

        # 판별(마스크 컴출모델)
        mask, nomask = model.predict(face_input).squeeze()

        # 마스크 착용 유무에 따른 라벨링
        if mask > nomask:
            color = (0, 255, 0)
            label = 'Mask %d%%' % (mask * 100)
        else:
            color = (0, 0, 255)
            label = 'No Mask %d%%' % (nomask * 100)

        # 화면에 얼굴부분과 마스크 유무를 출력해해줌
        cv2.rectangle(result_img, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
        cv2.putText(result_img, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8,
                    color=color, thickness=2, lineType=cv2.LINE_AA)

        
    time_tick += 1
    if time_tick % 5 == 1 :
        if mask<=nomask :
            playsound('bbi.mp3')
    cv2.imshow('VideoFrame', result_img)
    
    # q를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break
        

# 카메라 객체 반환
out.release()


# 화면에 나타난 윈도우들을 종료 ---> 실시간 영상
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6sxsq0tp\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
# 이전 시간 저장변수 하나 만들고
# 처음 돌 때 time.now() 로 현재시간 저장
# 이후로 돌 때마다 time.now()의 값과 이전 시간 저장변수 사이 차이 비교해서
# 5초 경과 시 아래 if mask~ 실행 및 이전 시간 저장변수 값을 현재 시간으로 업데이트